In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

In [ ]:
# Set the paths
data_dir = 'dataset-plant'
categories = ['Cerscospora', 'Healthy', 'Leaf rust', 'Miner', 'Phoma']

In [ ]:
# Load the images and labels
images = []
labels = []

In [ ]:
for category in categories:
    category_dir = os.path.join(data_dir, category)
    for filename in os.listdir(category_dir):
        img_path = os.path.join(category_dir, filename)
        img = Image.open(img_path)
        img = img.resize((128, 128))  # Resize the image to the desired size
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(categories.index(category))

In [ ]:
# Convert to numpy arrays
images = np.array(images, dtype='float32')
labels = np.array(labels)

In [ ]:
# Normalize the pixel values
images /= 255.0

In [ ]:
# Split the dataset into training and validation sets
x_train, x_val, _, _ = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
def load_data():
    return (x_train, None), (x_val, None)


In [ ]:
# Load and preprocess the data
(x_train, _), (x_val, _) = load_data()

In [ ]:
# Get the input shape
input_shape = x_train.shape[1:]  # Assuming the shape is (height, width, channels)


In [ ]:
# Define the input tensor
input_img = Input(shape=input_shape)

In [ ]:
# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(x)


In [ ]:
# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


In [ ]:
# Create the Autoencoder model
autoencoder = Model(input_img, decoded)


In [ ]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
# Print the model summary
autoencoder.summary()

In [ ]:
# Train the Autoencoder
autoencoder.fit(x_train, x_train, epochs=20, batch_size=128, validation_data=(x_val, x_val), shuffle=True)


In [ ]:
# Save the trained Autoencoder model
autoencoder.save('../aotoencoder_model/autoencoder.keras')